In [1]:
using Flux, Zygote, LinearAlgebra

In [2]:
ψmodel = Chain(
    Dense(2 => 16,tanh),
    Dense(16 => 16,tanh),
    # Dense(40 => 40,tanh),
    Dense(16 => 2,tanh)
)

Chain(
  Dense(2 => 16, tanh),                 # 48 parameters
  Dense(16 => 16, tanh),                # 272 parameters
  Dense(16 => 2, tanh),                 # 34 parameters
)                   # Total: 6 arrays, 354 parameters, 1.758 KiB.

In [3]:
ψmodel(randn(2,10))

2×10 Matrix{Float64}:
 -0.0125259  0.166454  0.0292023  …  -0.055071    0.0469169  -0.059211
  0.143239   0.154849  0.156331       0.00109607  0.174203   -0.017978

In [4]:
ψtest(xt) = (X -> ((x,t)->(collect∘reim)( sin(π*x)*exp(-1.0im*π^2*t/2) + sin(2π*x)*exp(-1.0im*π^2*2^2*t/2)))(X...)).(eachcol(xt))

ψtest (generic function with 1 method)

In [6]:
ψtest(xt) = sin.(π*xt[1,:]).*exp.(-1.0im*π^2*xt[2,:]/2) .+ sin.(2π*xt[1,:]).*exp.(-1.0im*π^2*2^2*xt[2,:]/2) |> X->(collect∘transpose∘hcat)(reim(X)...)

ψtest (generic function with 1 method)

In [7]:
ψmodel(randn(2,10))

2×10 Matrix{Float64}:
 -0.0758538  0.10046   0.0400557  …  0.038965  0.0277772  0.0228404
 -0.176918   0.120503  0.173477      0.172468  0.0811733  0.155802

In [8]:
collocationPts = collect([rand(10) 10rand(10)]');

In [5]:
Zygote.forward_jacobian(Y -> Zygote.jacobian(x->ψtest(x),Y)[1],collocationPts)

LoadError: UndefVarError: ψtest not defined

In [9]:
p,re = Flux.destructure(ψmodel)

(Float32[0.34468263, 0.20744629, 0.14022475, 0.15841107, 0.21398912, 0.18607108, 0.030785583, 0.38820067, 0.52960104, 0.5375387  …  -0.49819288, -0.30440265, 0.20408162, 0.2212743, 0.5674477, 0.20339426, 0.4632388, -0.21888323, 0.0, 0.0], Restructure(Chain, ..., 354))

In [10]:
function a_test(collocationPts,p,re)
    Dψ, DDψ = Zygote.jacobian(Y -> Zygote.jacobian(x->re(p)(x),Y)[1],collocationPts);
    # Dψ, DDψ = Zygote.forward_jacobian(Y -> Zygote.jacobian(x->re(p)(x),Y)[1],collocationPts);
    norm(Dψ)
end

a_test (generic function with 1 method)

In [18]:
function b_test(collocationPts,ψmodel)
    Dψ = Zygote.jacobian(Y -> Zygote.jacobian(x->ψmodel(x),Y)[1],collocationPts)[1];
    # Dψ, DDψ = Zygote.jacobian(Y -> Zygote.jacobian(x->ψmodel(x),Y)[1],collocationPts);
    # Dψ, DDψ = Zygote.forward_jacobian(Y -> Zygote.jacobian(x->re(p)(x),Y)[1],collocationPts);
    norm(Dψ)
end

b_test (generic function with 1 method)

In [19]:
Zygote.gradient(()->b_test(collocationPts,ψmodel),Flux.params(ψmodel))


In [11]:
a_test(collocationPts,p,re)

┌ Warning: second derivatives of Restructure may not work yet, sorry!
└ @ Optimisers /Users/emmyb320/.julia/packages/Optimisers/GKFy2/src/destructure.jl:158
┌ Warning: second derivatives of Restructure may not work yet, sorry!
└ @ Optimisers /Users/emmyb320/.julia/packages/Optimisers/GKFy2/src/destructure.jl:158
┌ Warning: second derivatives of Restructure may not work yet, sorry!
└ @ Optimisers /Users/emmyb320/.julia/packages/Optimisers/GKFy2/src/destructure.jl:158


LoadError: Mutating arrays is not supported -- called copyto!(SubArray{Float64, 1, Matrix{Float64}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}}, true}, ...)
This error occurs when you ask Zygote to differentiate operations that change
the elements of arrays in place (e.g. setting values with x .= ...)

Possible fixes:
- avoid mutating operations (preferred)
- or read the documentation and solutions for this error
  https://fluxml.ai/Zygote.jl/latest/limitations


In [ ]:
Zygote.jacobian(p->a_test(collocationPts,p,re),p)

┌ Warning: second derivatives of Restructure may not work yet, sorry!
└ @ Optimisers /Users/emmyb320/.julia/packages/Optimisers/GKFy2/src/destructure.jl:158
┌ Warning: second derivatives of Restructure may not work yet, sorry!
└ @ Optimisers /Users/emmyb320/.julia/packages/Optimisers/GKFy2/src/destructure.jl:158
┌ Warning: second derivatives of Restructure may not work yet, sorry!
└ @ Optimisers /Users/emmyb320/.julia/packages/Optimisers/GKFy2/src/destructure.jl:158


LoadError: Mutating arrays is not supported -- called setindex!(Matrix{Float64}, ...)
This error occurs when you ask Zygote to differentiate operations that change
the elements of arrays in place (e.g. setting values with x .= ...)

Possible fixes:
- avoid mutating operations (preferred)
- or read the documentation and solutions for this error
  https://fluxml.ai/Zygote.jl/latest/limitations


In [12]:
re(p)

Chain(
  Dense(2 => 40, tanh),                 # 120 parameters
  Dense(40 => 40, tanh),                # 1_640 parameters
  Dense(40 => 40, tanh),                # 1_640 parameters
  Dense(40 => 2, tanh),                 # 82 parameters
)                   # Total: 8 arrays, 3_482 parameters, 14.102 KiB.

In [13]:
using ForwardDiff;
const FD = ForwardDiff;

In [14]:
function a_test(collocationPts,p,re)
    DDψ = FD.jacobian(Y -> FD.jacobian(x->re(p)(x),Y),collocationPts);
    norm(DDψ)
end

a_test (generic function with 1 method)

In [15]:
a_test(collocationPts,p,re)

0.4229002591623485

In [17]:
using ReverseDiff
const RD = ReverseDiff

ReverseDiff

In [20]:
RD.gradient(p->a_test(collocationPts,p,re),p)

LoadError: InterruptException:

In [ ]:
function a_test(collocationPts,ψmodel)
    DDψ = FD.jacobian(Y -> FD.jacobian(x->re(p)(x),Y),collocationPts);
    norm(DDψ)
end

In [30]:
function b_test(collocationPts,ψmodel)
    Dψ, DDψ = Zygote.forward_jacobian(Y -> Zygote.jacobian(X->ψmodel(X),Y)[1],collocationPts);
    norm(DDψ)
end

b_test (generic function with 1 method)

In [32]:
b_test(collocationPts,ψmodel)

0.4229002591623486

In [33]:
Zygote.gradient(()->b_test(collocationPts,ψmodel),Flux.params(ψmodel))

LoadError: Mutating arrays is not supported -- called setindex!(Matrix{Float64}, ...)
This error occurs when you ask Zygote to differentiate operations that change
the elements of arrays in place (e.g. setting values with x .= ...)

Possible fixes:
- avoid mutating operations (preferred)
- or read the documentation and solutions for this error
  https://fluxml.ai/Zygote.jl/latest/limitations


: 

In [154]:
Zygote.jacobian(t->ψtest([collocationPts[1,1],t]),collocationPts[2,1])

([2.1106060931504373, 4.4614792075617515],)

In [149]:
Dψ

20×20 Matrix{Float64}:
 1.21378  2.11061   0.0         0.0       …    0.0      0.0        0.0
 6.16477  4.46148   0.0         0.0            0.0      0.0        0.0
 0.0      0.0       0.839997    0.120252       0.0      0.0        0.0
 0.0      0.0      -1.62302   -17.2143         0.0      0.0        0.0
 0.0      0.0       0.0         0.0            0.0      0.0        0.0
 0.0      0.0       0.0         0.0       …    0.0      0.0        0.0
 0.0      0.0       0.0         0.0            0.0      0.0        0.0
 0.0      0.0       0.0         0.0            0.0      0.0        0.0
 0.0      0.0       0.0         0.0            0.0      0.0        0.0
 0.0      0.0       0.0         0.0            0.0      0.0        0.0
 0.0      0.0       0.0         0.0       …    0.0      0.0        0.0
 0.0      0.0       0.0         0.0            0.0      0.0        0.0
 0.0      0.0       0.0         0.0            0.0      0.0        0.0
 0.0      0.0       0.0         0.0            0.0    

In [16]:
function a_test
Dψ, DDψ = Zygote.forward_jacobian(Y -> Zygote.jacobian(x->ψmodel(x),Y)[1],randn(2,10));

In [83]:
[sum([DDψ .!= 0.0][1][:,1:2 + 2i]) for i in 0:19]

20-element Vector{Int64}:
 4
 4
 4
 4
 4
 4
 4
 4
 4
 4
 8
 8
 8
 8
 8
 8
 8
 8
 8
 8

In [9]:
Zygote.forward_jacobian(x->ψmodel(x),randn(2))

([-0.4647458375393646, 0.40574901550852277], [0.12378217399477319 -0.18787819195596278; -0.132936019452642 -0.08764034973969359])

In [7]:
Zygote.forward_jacobian(Y -> Zygote.jacobian(x->ψmodel(x),Y)[1],randn(2,1))

([0.68461640813541 -0.6527095228649112; -0.29970379221004206 0.12806127706895734], [-0.09740624351262038 -0.03758997067101796 0.14113348270538417 -0.023942858537649415; 0.1411334827053841 -0.02394285853764943 -0.2333561415022148 0.021241809901526677])